In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go

import data

In [12]:
# Solve ODE
time_span: tuple[float, float] = (0, data.SIMULATION_TIME)  # Time span in seconds
initial_conditions: list[float] = [data.T_METAL_0, data.T_WATER_0]

solution = solve_ivp(
    data.temperature_ode,
    time_span,
    initial_conditions,
    t_eval=np.linspace(0, data.SIMULATION_TIME, 1000),
    method="LSODA",
)

In [13]:
# Extract results
time = solution.t
T_metal = solution.y[0]
T_water = solution.y[1]

_, fraction = data.boiling_heat_flux_vect(T_metal)

In [15]:
# Create figure
fig = go.Figure()

# Add Metal Temperature trace
fig.add_trace(
    go.Scatter(x=time, y=T_metal - 273.15, mode="lines", name="Metal Temperature")
)

# Add Water Temperature trace
fig.add_trace(
    go.Scatter(x=time, y=T_water - 273.15, mode="lines", name="Water Temperature")
)

# Add Saturation Temperature line
fig.add_trace(
    go.Scatter(
        x=[time[0], time[-1]],
        y=[data.T_SAT - 273.15] * 2,
        mode="lines",
        name="Saturation Temperature",
        line=dict(color="red", dash="dash"),
    )
)

# Add Boiling Intensity trace on secondary y-axis
fig.add_trace(
    go.Scatter(
        x=time,
        y=fraction,
        mode="lines",
        name="Boiling Intensity (right axis)",
        yaxis="y2",
    )
)

# Add vertical dashed line at the end of heating
fig.add_trace(
    go.Scatter(
        x=[data.TURNED_OFF_TIME, data.TURNED_OFF_TIME],
        y=[min(T_metal - 273.15), max(T_metal - 273.15)],
        mode="lines",
        name=f"Heating Stops @ {data.TURNED_OFF_TIME} s",
        line=dict(color="black", dash="dash", width=1),
    )
)

# Update layout
fig.update_layout(
    title="Transient Temperature Evolution during Water Boiling",
    xaxis=dict(title="Time (s)"),
    yaxis=dict(title="Temperature (°C)"),
    yaxis2=dict(title="Boiling Intensity (-)", overlaying="y", side="right"),
    legend=dict(x=0.5, y=0.1),
    template="plotly_white",
)

# Show the figure
fig.show()